In [1]:
%load_ext autoreload
%autoreload 2
import mmcv
from mmdet3d.apis import inference_mono_3d_detector, init_model
from mmdet3d.registry import VISUALIZERS
import pickle
from mmdet3d.registry import VISUALIZERS
import cv2
import numpy as np
import sys

sys.path.append('../tools')

from data_processor import DataProcessor, create_pkl
from visualizer import ImageVisualizer, draw_monodetection_labels, draw_monodetection_results, LidarVisualizer, draw_monorestults_in_lidar

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [1]:
file = '002238'
dataset = 'dair'
if dataset == 'kitti':
    img_file = '/home/javier/datasets/kitti/training/image_2/' + file + '.png'
    labels_file = '/home/javier/datasets/kitti/training/label_2/'+ file + '.txt'
    calib_file = '/home/javier/datasets/kitti/training/calib/'+ file + '.txt'
    pcd_file = '/home/javier/datasets/kitti/training/velodyne/' + file +'.bin'
    pitch = 0.0
elif dataset == 'dair':
    img_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/image_2/' + file + '.png'
    labels_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/label_2/'+ file + '.txt'
    calib_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/calib/'+ file + '.txt'
    pcd_file = '/home/javier/datasets/DAIR/single-infrastructure-side-mmdet/training/velodyne/' + file +'.bin'
    pitch = 0.2031

ann_file = '/home/javier/sensus-loci/sensus/notebooks/002238.pkl'
cam_type = 'CAM_BACK'

## Create PKL file

In [3]:
create_pkl(file, calib_file, cam_type, ann_file)

Data saved to /home/javier/sensus-loci/sensus/notebooks/002238.pkl


## Inference

In [4]:
config_fcos = '/home/javier/sensus-loci/sensus/configs/smoke/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d.py'

checkpoint_fcos = '/home/javier/sensus-loci/work_dirs/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d/epoch_100.pth'

device = 'cuda:0'
model_mmdet = init_model(config_fcos, checkpoint_fcos, device=device)
result = inference_mono_3d_detector(model_mmdet, img_file, ann_file, cam_type)

# init visualizer
visualizer_mmdet = VISUALIZERS.build(model_mmdet.cfg.visualizer)
visualizer_mmdet.dataset_meta = model_mmdet.dataset_meta

Loads checkpoint by local backend from path: /home/javier/sensus-loci/work_dirs/smoke_dla34_dlaneck_gn-all_4xb8-6x_dair-mono3d/epoch_100.pth
09/26 16:36:20 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_0.projs.0.conv is upgraded to version 2.
09/26 16:36:20 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_0.nodes.0.conv is upgraded to version 2.
09/26 16:36:20 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.projs.0.conv is upgraded to version 2.
09/26 16:36:20 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.projs.1.conv is upgraded to version 2.
09/26 16:36:20 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.nodes.0.conv is upgraded to version 2.
09/26 16:36:20 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_1.nodes.1.conv is upgraded to version 2.
09/26 16:36:20 - mmengine - INFO - ModulatedDeformConvPack neck.dla_up.ida_2.projs.0.conv is upgraded to version 2.
09/26 16:36:20 - mmengine - INFO - ModulatedDef

/home/javier/miniconda3/envs/openmmlab/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


## Sensus Viz

In [5]:
print(result.pred_instances_3d.scores_3d)

tensor([0.5217, 0.4985, 0.4795, 0.4576, 0.4507, 0.4311, 0.4160, 0.4013, 0.3731,
        0.3315, 0.3309, 0.3294, 0.3123, 0.3022], device='cuda:0')


In [6]:
draw_monodetection_results(img_file, calib_file, result, score=0.41, pitch=pitch, thickness=1)

[1.542670726776123, 1.8651961088180542, 4.262558460235596]
[1.5057939291000366, 1.856995701789856, 4.242576599121094]
[1.4859989881515503, 1.8845899105072021, 4.385947227478027]
[2.678711414337158, 2.51883602142334, 6.373397350311279]
[1.727476954460144, 1.9132280349731445, 4.239100456237793]
[1.7327828407287598, 1.9342377185821533, 4.45134162902832]
[1.8245452642440796, 1.9512022733688354, 4.261423587799072]


## Compare results

In [7]:
draw_monorestults_in_lidar(pcd_file, calib_file, result, img_file, labels_file)

[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] ICE servers: {"stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"}
FEngine (64 bits) created at 0x7fa0300079e0 (threading is enabled)
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.
EGL(1.5)
OpenGL(4.1)


WebVisualizer(window_uid='window_0')